#### packages and functions

In [3]:
from pathlib import Path
import numpy as np 
import h5py
import matplotlib.pyplot as plt
import time
import os
from scipy import signal
import matplotlib.colors as colors
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import axes3d
from scipy.optimize import curve_fit
from scipy.special import erf
import scipy.io
import colorsys
from scipy import signal
import math
from scipy.stats import norm
import pickle
import os
import pandas as pd

import ipyparams


## Load data

### User input section

In [4]:
########################### User input section ############################
# run info
currentnotebook_name = ipyparams.notebook_name
run_name = currentnotebook_name[:-6]
run_name = 'run0'

#Boolean input
override_num_shots = False
reset_hard = False #only set this to true if you want to reload from raw data


#number input
# num_shots_manual = 39
num_shots_manual = 500
num_frames = 3
outer_zoom_factor = 5
point_name_outer = 'sideprobe_intensity'
point_name_inner = 'cav_pzt'

point_list_outer = np.array([1.7]) #ControlV_Power(np.array([1,1.2,1.4,1.6,1.8,2,2.5,4]))
point_list_inner = np.linspace(-0.6,0.6,7)

# point_list_inner = np.linspace(0.35,0.5,4)

tweezer_freq_list = 88 + 0.8*0 + 0.8*np.arange(40)

twz_num_plot=np.array([18])


atom_site = []
for i in range(num_frames):
    atom_site.append(np.arange(len(tweezer_freq_list)))

#################################################################################
num_points_inner = len(point_list_inner)
num_points_outer = len(point_list_outer)
if num_points_outer == 1:
    point_list = np.array(point_list_inner)
else:
    point_list = (outer_zoom_factor*np.outer(point_list_outer,np.ones(len(point_list_inner))) + np.outer(np.ones(len(point_list_outer)),point_list_inner)).flatten()
    plt.plot(point_list)
num_points = len(point_list)


### load from GaGe scope

In [8]:
reset_gage = reset_hard
# reset_gage = True
time_me = True
plot_tenth_shot = True
het_freq = 20.000446 #MHz
# het_freq = 20.000 #MHz
dds_freq = het_freq/2
samp_freq = 200 #MHz
# averaging_time = 100 #us
# step_time = 5 #us
# filter_time = 10 #us
step_time = 50 #us
filter_time = 100 #us
voltage_conversion = 1000/32768 #in units of mV
kappa = 2*np.pi * 1.1 #MHz
LO_power = 314 #uW
PHOTON_ENERGY = 2.55e-19
LO_rate = 1e-12 * LO_power / PHOTON_ENERGY # count/us
photonrate_conversion = 1e-6/(2e7)/ PHOTON_ENERGY / (0.5*0.8) # count/us
#2e7 is the conversion gain, 2.55e-19 is single photon energy, the rest is the path efficiency
heterodyne_conversion = 1/np.sqrt(LO_rate) # np.sqrt(count/us)
cavity_conversion = 1/np.sqrt(kappa)
conversion_factor = voltage_conversion*photonrate_conversion*heterodyne_conversion*cavity_conversion
# filter_type = "square"
num_shot_gage_start = 0
num_shots_gage = 1
if override_num_shots:
    num_shots_gage = 1
    
start = time.time()
try:
    num_shots_loaded = np.load(f'{run_name}_gage_cmplx_amp_{filter_time}_{step_time}.pkl', allow_pickle=True).shape[1]
except FileNotFoundError:
    num_shots_loaded = 0
try:
    np.load(f'{run_name}_gage_timebin_{filter_time}_{step_time}.pkl', allow_pickle=True)
except FileNotFoundError:
    num_shots_loaded = 0
if time_me:
    file_reading_time = 0
    inner_product_time = 0
    now = start
#################################################################
if reset_gage:
#     path, dirs, files = next(os.walk(data_path_gage))
#     num_shot_gage_start = 0
#     num_shots_gage = len(files)
# #     num_shots = num_shots_gage
#     if override_num_shots:
#         num_shots_gage = num_shots_manual
    print(f'Hard reset, loading {num_shots_gage} shots from gage raw data')
    hf = h5py.File("gage_shot_00000.h5", 'r')
    chlen = len(np.array(hf.get('CH1')))
    t_vec = np.arange(chlen)*1/samp_freq
    ch1_pure_vec = np.exp(-1j*2*np.pi * dds_freq*t_vec)
    ch3_pure_vec = np.exp(-1j*2*np.pi * het_freq*t_vec)
#                 t0_list=np.linspace(0,len(ch1)/samp_freq-filter_time,int(len(ch1)/samp_freq/step_time))
    t0_list = np.arange(0,chlen/samp_freq-filter_time+step_time,step_time)
    timebin_array = np.empty((len(t0_list),2), dtype=float)
    timebin_array[:,0] = t0_list
    timebin_array[:,1] = t0_list + filter_time
    cmplx_amp_array = np.empty((2, num_shots_gage,len(t0_list)), dtype=np.cdouble)
    t0_idx_list = np.arange(0,chlen-filter_time*samp_freq+1,step_time*samp_freq)
#     t1_idx_list = t0_idx_list + filter_time*samp_freq
    for shot_num in range(num_shots_gage):
        if shot_num%100 == 0:
            print(f'shot{shot_num} done')
        if True: #mask_valid_data[shot_num]:
            hf = h5py.File("gage_shot_00000.h5", 'r')
#             ch1 = np.array(hf.get('CH1')) * voltage_conversion
#             ch3 = np.array(hf.get('CH3')) * voltage_conversion
            ch1 = np.array(hf['CH1']) * conversion_factor #np.sqrt(n)
            ch3 = np.array(hf['CH3']) * conversion_factor #np.sqrt(n)
            if time_me:
                file_reading_time += time.time() - now
                now = time.time()
            ch1_demod = np.multiply(ch1 , ch1_pure_vec) # * 1j
            ch3_demod = np.multiply(ch3 , ch3_pure_vec) # * 1j
            ch1_demod_sum = np.concatenate([[0],np.cumsum(ch1_demod)])
            ch3_demod_sum = np.concatenate([[0],np.cumsum(ch3_demod)])
            cmplx_amp_list_ch1=t0_list*0j
            cmplx_amp_list_ch3=t0_list*0j
            for i,t0_idx in enumerate(t0_idx_list):
                t1_idx = t0_idx + filter_time*samp_freq
#                 cmplx_amp_list_ch1[i] = np.mean(ch1_demod[t0_idx:t1_idx])
#                 cmplx_amp_list_ch3[i] = np.mean(ch3_demod[t0_idx:t1_idx])
                cmplx_amp_list_ch1[i] = (ch1_demod_sum[t1_idx] - ch1_demod_sum[t0_idx]) / (t1_idx-t0_idx)
                cmplx_amp_list_ch3[i] = (ch3_demod_sum[t1_idx] - ch3_demod_sum[t0_idx]) / (t1_idx-t0_idx)
            if time_me:
                inner_product_time += time.time() - now
                now = time.time()
            cmplx_amp_array[0, shot_num] = cmplx_amp_list_ch1
            cmplx_amp_array[1, shot_num] = cmplx_amp_list_ch3
            if plot_tenth_shot and shot_num == 50:
                t_start = 200
                amp_list_ch1 = abs(cmplx_amp_list_ch1)
                angle_list_ch1 = np.angle(cmplx_amp_list_ch1)%(2*np.pi)
                amp_list_ch3 = abs(cmplx_amp_list_ch3)
                angle_list_ch3 = np.angle(cmplx_amp_list_ch3)%(2*np.pi)
                fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(16,5))
                ax1.plot(np.arange(0,filter_time,1/samp_freq), ch1[0:filter_time*samp_freq])
                ax2.plot(np.arange(0,filter_time,1/samp_freq), ch3[0:filter_time*samp_freq])
                ax1.set_ylabel("mV")
                ax2.set_ylabel("mV")
                plt.show()
                fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(16,5))
                ax1.plot(t0_list,(2*np.unwrap(angle_list_ch1)))
                ax1.plot(t0_list,np.unwrap(angle_list_ch3))
                ax2.plot(t0_list,amp_list_ch1)
                ax2.plot(t0_list,amp_list_ch3)
                ax1.set_xlabel("us")
                ax2.set_xlabel("us")
                plt.show()
        else:
            cmplx_amp_array[:, shot_num, :] = np.array([np.nan])
    with open(f'{run_name}_gage_cmplx_amp_{filter_time}_{step_time}.pkl','wb') as f1:
         pickle.dump(cmplx_amp_array, f1)
    with open(f'{run_name}_gage_timebin_{filter_time}_{step_time}.pkl','wb') as f3:
         pickle.dump(timebin_array, f3)
    print('done')
    if time_me:
        print(f"file_reading_time = {file_reading_time} s")
        print(f"inner_product_time = {inner_product_time} s")
####################################################################
else:
    print(f'loading {num_shots_loaded} shots from gage pickle files')
    try:
        cmplx_amp_array_old = np.load(f'{run_name}_gage_cmplx_amp_{filter_time}_{step_time}.pkl', allow_pickle=True)
        timebin_array = np.load(f'{run_name}_gage_timebin_{filter_time}_{step_time}.pkl', allow_pickle=True)
    except:
        print('first time run')
#         path, dirs, files = next(os.walk(data_path_gage))
#         num_shots_gage = len(files)
#         if override_num_shots:
#             num_shots_gage = num_shots_manual
####################################################################
    if (num_shots_gage > num_shots_loaded):
        print(f'loading {num_shots_loaded} to {num_shots_gage} shots from gage raw data')
        hf = h5py.File("gage_shot_00000.h5", 'r')
        chlen = len(np.array(hf.get('CH1')))
        t_vec = np.arange(chlen)*1/samp_freq
        ch1_pure_vec = np.exp(-1j*2*np.pi * dds_freq*t_vec)
        ch3_pure_vec = np.exp(-1j*2*np.pi * het_freq*t_vec)
    #                 t0_list=np.linspace(0,len(ch1)/samp_freq-filter_time,int(len(ch1)/samp_freq/step_time))
        t0_list = np.arange(0,chlen/samp_freq-filter_time+step_time,step_time)
        timebin_array = np.empty((len(t0_list),2), dtype=float)
        timebin_array[:,0] = t0_list
        timebin_array[:,1] = t0_list + filter_time
        cmplx_amp_array = np.empty((2, num_shots_gage,len(t0_list)), dtype=np.cdouble)
        if num_shots_loaded > 0:
            cmplx_amp_array[:,0:num_shots_loaded,:] = cmplx_amp_array_old
        t0_idx_list = np.arange(0,chlen-filter_time*samp_freq+1,step_time*samp_freq)
    #     t1_idx_list = t0_idx_list + filter_time*samp_freq
        for shot_num in range(num_shots_loaded, num_shots_gage):
            if shot_num%100 == 0:
                print(f'shot{shot_num} done')
            if True:
                hf = h5py.File("gage_shot_00000.h5", 'r')
    #             ch1 = np.array(hf.get('CH1')) * voltage_conversion
    #             ch3 = np.array(hf.get('CH3')) * voltage_conversion
                ch1 = np.array(hf['CH1']) * conversion_factor
                ch3 = np.array(hf['CH3']) * conversion_factor
                if time_me:
                    file_reading_time += time.time() - now
                    now = time.time()
                ch1_demod = np.multiply(ch1 , ch1_pure_vec) # * 1j
                ch3_demod = np.multiply(ch3 , ch3_pure_vec) # * 1j
                ch1_demod_sum = np.concatenate([[0],np.cumsum(ch1_demod)])
                ch3_demod_sum = np.concatenate([[0],np.cumsum(ch3_demod)])
                cmplx_amp_list_ch1=t0_list*0j
                cmplx_amp_list_ch3=t0_list*0j
                for i,t0_idx in enumerate(t0_idx_list):
                    t1_idx = t0_idx + filter_time*samp_freq
                    cmplx_amp_list_ch1[i] = (ch1_demod_sum[t1_idx] - ch1_demod_sum[t0_idx]) / (t1_idx-t0_idx)
                    cmplx_amp_list_ch3[i] = (ch3_demod_sum[t1_idx] - ch3_demod_sum[t0_idx]) / (t1_idx-t0_idx)
                if time_me:
                    inner_product_time += time.time() - now
                    now = time.time()
                cmplx_amp_array[0, shot_num] = cmplx_amp_list_ch1
                cmplx_amp_array[1, shot_num] = cmplx_amp_list_ch3
            else:
                cmplx_amp_array[:, shot_num, :] = np.array([np.nan])
        with open(f'{run_name}_gage_cmplx_amp_{filter_time}_{step_time}.pkl','wb') as f1:
             pickle.dump(cmplx_amp_array, f1)
        with open(f'{run_name}_gage_timebin_{filter_time}_{step_time}.pkl','wb') as f3:
             pickle.dump(timebin_array, f3)
        print('done')
    else:
        cmplx_amp_array=cmplx_amp_array_old
####################################################################
print(f"number of {num_shots_gage} shots loaded")
print(f"total time elapsed {time.time()-start} s")

loading 0 shots from gage pickle files
first time run
loading 0 to 1 shots from gage raw data
shot0 done
done
number of 1 shots loaded
total time elapsed 0.2318253517150879 s
